SARIMA Evaluation: AIC, BIC, MAPE, MAE

In [ ]:
#SARIMA Evaluation: AIC, BIC, MAPE, MAE

from statsmodels.tools.eval_measures import mse, meanabs
from sklearn.metrics import mean_absolute_percentage_error

# Assume you have a SARIMA results model and test data
# Let's split last 30 days for testing
train = df['liquidity'][:-30]
test = df['liquidity'][-30:]

# Refit model on training
from src.sarima_model import train_sarima
sarima_model = train_sarima(train)

# Forecast
forecast = sarima_model.get_forecast(steps=30).predicted_mean

# Evaluation metrics
aic = sarima_model.aic
bic = sarima_model.bic
mape = mean_absolute_percentage_error(test, forecast)
mae = meanabs(test, forecast)

print("SARIMA Evaluation:")
print(f"AIC: {aic:.2f}")
print(f"BIC: {bic:.2f}")
print(f"MAPE: {mape:.4f}")
print(f"MAE: {mae:.2f}")


SARIMA Evaluation: AIC, BIC, MAPE, MAE

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

from src.xgboost_model import prepare_features, train_xgb

df_feat = prepare_features(df.copy())

X = df_feat[[f'lag_{i}' for i in range(1, 6)]]
y = df_feat['liquidity']

# Simple 80/20 split
split_idx = int(0.8 * len(df_feat))
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

# Train and predict
xgb_model = train_xgb(pd.concat([X_train, y_train], axis=1))
preds = xgb_model.predict(X_test)

# Metrics
rmse = np.sqrt(mean_squared_error(y_test, preds))
mae = mean_absolute_error(y_test, preds)

print("\nXGBoost Evaluation:")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")


 Difference-in-Differences (DiD) Evaluation

In [ ]:
from src.causal_inference import run_did

# Already contains 'event', 'treated', 'interaction'
did_model = run_did(df.copy())

# Look at p-value for the interaction term
interaction_pval = did_model.pvalues['interaction']
interaction_coef = did_model.params['interaction']

print("\nDiD Evaluation:")
print(f"Interaction Coefficient: {interaction_coef:.2f}")
print(f"p-value: {interaction_pval:.4f}")
if interaction_pval < 0.05:
    print("✅ Statistically significant impact of market event on liquidity.")
else:
    print("❌ No statistically significant impact.")
